In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D,LSTM, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os 
%matplotlib inline

In [2]:
# 检查可用GPU数量
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSH/SSH_2005-2015_month.nc')

In [4]:
time_ssh = data_ssh['time'][:]
print(len(time_ssh))

132


In [5]:
# 查看研究区域经纬度
ssh_lat = data_ssh['lat'][:].data
#print(ssh_lat[:])
ssh_lon = data_ssh['lon'][:].data
#print(ssh_lon[:])

In [6]:
ssh = data_ssh['ssh'][:].data
print(ssh.shape)

(132, 280, 600)


In [7]:
np.nanmax(ssh),np.nanmin(ssh)

(2.20051, -0.43673334)

## SSS

In [8]:
data_sss = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSS/dataset-sss-ssd-rep-monthly_2005-2020.nc')

In [9]:
# 确定时间
time_sss = data_sss['time'][:132]
print(len(time_sss))

132


In [10]:
# 获取研究位置经纬度
sss_lat = data_sss['lat'][:].data
#print(sss_lat[:])
sss_lon = data_sss['lon'][:].data
#print(sss_lon[:])

In [11]:
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss) # 移除大小为一的维度
print(sss.shape)

(132, 1, 280, 600)
(132, 280, 600)


In [12]:
np.nanmax(sss),np.nanmin(sss)

(36.072178, 13.604756)

## SSW

In [13]:
data_ssw = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/SSW/anon_ssw_2005_2018_Month_data.nc')

In [14]:
# 确定时间
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))

132


In [15]:
lat_ssw = data_ssw['lat'][:].data
#print(lat_ssw[314:594])
lon_ssw = data_ssw['lon'][:].data
#print(lon_ssw[440:1040])

# lat 0.125    ----    69.875
# lon  110.125    ----    259.875

In [16]:
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data

In [17]:
print(uwnd.shape)
print(vwnd.shape)

(132, 280, 600)
(132, 280, 600)


In [18]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## 3DT

In [19]:
data_3dt_1 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2005-2009.nc')
data_3dt_2 = nc.Dataset(r'D:/codeFile/jupyterDemo/3d_ts/BPNN/data/3DT/dataset-armor-3d-rep-monthly_2010-2015.nc')

In [20]:
sst_1 = data_3dt_1['to'][:].data
sst_2 = data_3dt_2['to'][:].data

In [21]:
print(sst_1.shape)
print(sst_2.shape)

(60, 43, 280, 600)
(72, 43, 280, 600)


###  30 - 2000m 温度数据

In [22]:
temp_depth = np.append(sst_1,sst_2,axis=0)
print(temp_depth.shape)

(132, 43, 280, 600)


In [23]:
# 所有深度层
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]

#选取的深度层
depths_use = [10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]

#[50,100,125,150,200,250,300,400,500,600,700,800,900,1000]

In [24]:
temp_depth_use = temp_depth[:,1:2]

In [25]:
for i,depth in enumerate(depths):
    if(depth in depths_use):
        #print(depths[i])
        temp_depth_use = np.concatenate((temp_depth_use,temp_depth[:,i:i+1]),axis=1)  

In [26]:
print(temp_depth_use.shape)
temp_depth_use = np.transpose(temp_depth_use, (0,2,3,1))
print(temp_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## SST表面温度数据

In [27]:
sst = temp_depth[:,0:1,:,:]

In [28]:
sst = np.squeeze(sst)
print(sst.shape)

(132, 280, 600)


## 数据合并

In [29]:
print(ssh.shape,sss.shape,uwnd.shape,vwnd.shape,sst.shape)
cat_data =np.stack((ssh,sss,uwnd,vwnd,sst),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [30]:
# 将表面数据和水下数据在最后一个维度拼接，将空值删除。
cat_data_ss_depth = np.concatenate((cat_data,temp_depth_use),axis=-1)

In [31]:
cat_data_ss_depth.shape 

(132, 280, 600, 38)

## 将两个温度异常区域的数据设置为nan

In [32]:
test = temp_depth_use[:,:,:,27:28]
test[test < 0.4] = np.nan
test[test > 10] = np.nan

In [33]:
cat_data_ss_depth[:,:,:,32:33] = test

In [34]:
cat_data_ss_depth.shape 

(132, 280, 600, 38)

## 将填充值赋值为NAN，且一个点中有一个维度为nan，则将这个点的所有维度都设置为nan 


In [35]:
## 先对合并的数据进行reshape成（长*宽，时间*通道），此步骤是为了统一图像， 让所有时间所有深度的图像都相同。
cat_data_ss_depth_reshape  = np.transpose(cat_data_ss_depth,(1,2,0,3))
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape ((-1,132*38))
print(cat_data_ss_depth_reshape.shape)

(280, 600, 132, 38)
(168000, 5016)


In [36]:
# 将填充值赋值为nan
cat_data_ss_depth_reshape[cat_data_ss_depth_reshape == 32767] = np.nan
nan_mask = np.isnan(cat_data_ss_depth_reshape)

In [38]:
# 查看nan的个数 64471
np.sum(nan_mask.any(axis=1) == True)

64471

In [39]:
# 通过布尔索引删除NaN值。
cat_data_ss_depth_reshape[nan_mask.any(axis=1) == True] = np.nan   
# nan_mask.any(axis=1)==true,只要第一个维度中含有nan就将所有设置为nan。
print('cat_data_ss_depth_reshape:',cat_data_ss_depth_reshape.shape)

cat_data_ss_depth_reshape: (168000, 5016)


In [40]:
# 查看nan的个数，共有64471。
np.sum(np.isnan(cat_data_ss_depth_reshape[:,4])) # 此处写几都可以，得到的nan的数量都是相同的。 

64471

In [41]:
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,132,38)
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = np.transpose(cat_data_ss_depth_reshape,(1,0,2))
print(cat_data_ss_depth_reshape.shape)

(168000, 132, 38)
(132, 168000, 38)


In [42]:
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,280,600,38)
print(cat_data_ss_depth_reshape.shape)

(132, 280, 600, 38)


In [43]:
# 确认nan的个数
np.sum(np.isnan(cat_data_ss_depth_reshape[1:2,:,:,2:3]))

64471

## 将数据分成小块

In [44]:
cat_data_ss = cat_data_ss_depth_reshape[:,:,:,:5]    #表面数据
cat_data_depth = cat_data_ss_depth_reshape[:,:,:,5:]  #次表层数据
print(cat_data_ss.shape,cat_data_depth.shape)

(132, 280, 600, 5) (132, 280, 600, 33)


In [45]:
np.sum(np.isnan(cat_data_depth[1:2,:,:,5:6]))

64471

In [46]:
import numpy as np
import pandas as pd

time_size, lat_size, lon_size, channel = cat_data_ss.shape   # （132，280，600，5）

# 定义矩形框的大小和步幅           
box_size = 9  # 单位为度
step_size = 1  # 滑动步幅

# 计算输出网格的形状
out_lat_size = (lat_size - box_size) // step_size  + 1 
out_lon_size = (lon_size - box_size) // step_size  + 1 

print('time_size:',time_size,'lat_size:',lat_size,'lon_size:',lon_size)   #(132 280 600)
print(out_lat_size,out_lon_size)  #(272 592)

#输出数组 
out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) #(132, 272, 592, 9,9,5)
out_data_y = np.zeros((time_size, out_lat_size , out_lon_size, 33)) #(132, 272, 592,15)

# 遍历时间步、纬度和经度
for t in range(time_size):
    sample_idx = 0
    for lat in range(0, lat_size - box_size +1, step_size):
        for lon in range(0, lon_size - box_size +1, step_size):
            # 计算当前矩形框的边界
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            #print(lat_min,lat_max,lon_min,lon_max)
            # 获取当前矩形框的地图数据
            box_data = cat_data_ss[t, lat_min:lat_max, lon_min:lon_max,:]
            # 写入数组中
#             lat_ = lat//4
#             lon_ = lon//4
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []
            #print(sample_idx)
            # 增加样本计数器
            sample_idx += 1

print('out_data的形状',out_data.shape)
#return out_data
    
#==============================================================================#   

# 遍历时间步、纬度和经度
for t in range(time_size):
    sample_idx = 0
    for lat in range(4, lat_size-4, step_size):
        for lon in range(4, lon_size-4, step_size):
            box_data_y = cat_data_depth[t, lat:lat+1, lon:lon+1,:]
            out_data_y[t,lat-4,lon-4, :] = box_data_y
            box_data_y = []
            #print(sample_idx)
            # 增加样本计数器
            sample_idx += 1

print('out_data_y的形状',out_data_y.shape)
    
# out_data_reshape = slider_value(cat_data)

time_size: 132 lat_size: 280 lon_size: 600
272 592
out_data的形状 (132, 272, 592, 9, 9, 5)
out_data_y的形状 (132, 272, 592, 33)


### test 

In [47]:
np.sum(np.isnan(out_data_y[1:2,:,:,2:3]))  # 之前的nan值个数为61950，当前为57599

59992

In [48]:
#out_data[1,24,1,:,:,4]

In [49]:
#out_data_y[1,24,1,0]

## 删除缺失值多于一半的数据

### 获取NAN_mask 

In [50]:
out_data.shape 

(132, 272, 592, 9, 9, 5)

In [51]:
# 修改变量形状
out_data_reshape = out_data.reshape((132,-1,9,9,5))
print(out_data_reshape.shape) 

# 获取变量nan掩码
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

(132, 161024, 9, 9, 5)
(132, 161024, 9, 9, 5)


In [54]:
# （161024，9，9）中存在4666932个nan
np.count_nonzero(nan_mask_out_data[7:8,:,:,:,2:3])

4861287

In [52]:
# 选取一个作为代表(都一样)
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,0]   #(161024, 9, 9)
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) #（161024，81）    # 其中true 代表Nan

In [55]:
# 查看mask中Nan 的个数
#np.count_nonzero(nan_mask_out_data_one)     # 4666932
#查看形状
#nan_mask_out_data_one.shape     # 161024, 81)
# 查看内容
#nan_mask_out_data_one

### 为有效值大于一半的方块标注True

In [56]:
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40):    #True 代表nan，当nan小于等于40时，代表有效值大于一半
        nan_mask.append(True)
    else:
        nan_mask.append(False)

In [57]:
np.count_nonzero(nan_mask)  # 查看有效值个数

101392

In [58]:
out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
print('out_data_y_reshape:',out_data_y_reshape.shape)
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,33)
print('out_data_y_reshape:',out_data_y_reshape.shape)

out_data_reshape: (161024, 132, 9, 9, 5)
out_data_y_reshape: (272, 592, 132, 33)
out_data_y_reshape: (161024, 132, 33)


## 开始删除无效数据

In [59]:
out_data_reshape = out_data_reshape[nan_mask]
out_data_y_reshape = out_data_y_reshape[nan_mask]
print(out_data_reshape.shape,out_data_y_reshape.shape)

(101392, 132, 9, 9, 5) (101392, 132, 33)


## 使用0来填充nan 

In [60]:
out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)   # (103623, 132, 9, 9, 5) 
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  #(103623, 132, 15)

# 验证集和测试集的划分

## 对数据进行reshape,以进行数据集的划分

In [61]:
out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)

out_data_reshape: (132, 101392, 9, 9, 5)
out_data_y_reshape: (132, 101392, 33)


In [62]:
# 最后十二个月的数据作为test
test_num = 120
x_train = out_data_reshape[:test_num]
y_train = out_data_y_reshape[:test_num]

x_test = out_data_reshape[test_num:]
y_test = out_data_y_reshape[test_num:]

In [63]:
# 查看形状
x_train.shape,y_train.shape,x_test.shape,y_test.shape 

((120, 101392, 9, 9, 5),
 (120, 101392, 33),
 (12, 101392, 9, 9, 5),
 (12, 101392, 33))

In [64]:
# 将前两个维度进行合并  时间*(lat*lon)
x_train = np.reshape(x_train,(-1,9,9,5))
x_test = np.reshape(x_test,(-1,9,9,5))

y_train = np.reshape(y_train,(-1,33))
y_test = np.reshape(y_test,(-1,33))

## 划分训练数据和目标数据

In [65]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train',x_train.shape)
print('y_train',y_train.shape)

x_test.shape: (1216704, 9, 9, 5)
y_test.shape (1216704, 33)
x_train (12167040, 9, 9, 5)
y_train (12167040, 33)


In [66]:
# 这是验证集和训练集的划分是随机选取。
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=100)

In [67]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('x_val',x_val.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_val',y_val.shape)

x_train (8516928, 9, 9, 5)
x_test (1216704, 9, 9, 5)
x_val (3650112, 9, 9, 5)
y_train (8516928, 33)
y_test (1216704, 33)
y_val (3650112, 33)


# 特征归一化

In [68]:
# 记录数据集的形状
x_train_shape = x_train.shape
y_train_shape = y_train.shape

x_test_shape = x_test.shape
y_test_shape = y_test.shape

x_val_shape = x_val.shape
y_val_shape = y_val.shape

In [69]:
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train.reshape(-1,x_train_shape[3])) 
x_test = scaler_f.transform(x_test.reshape(-1,x_test_shape[3]))
x_val = scaler_f.transform(x_val.reshape(-1,x_val_shape[3]))

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train.reshape(-1,y_train_shape[1])) 
y_test = scaler_l.transform(y_test.reshape(-1,y_test_shape[1])) 
y_val = scaler_l.transform(y_val.reshape(-1,y_val_shape[1])) 

In [70]:
x_train = np.reshape(x_train,(-1,9,9,5))
x_test = np.reshape(x_test,(-1,9,9,5))
x_val = np.reshape(x_val,(-1,9,9,5))
y_train = np.reshape(y_train,(-1,33))
y_test = np.reshape(y_test,(-1,33))
y_val = np.reshape(y_val,(-1,33))

In [71]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('x_val',x_val.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_val',y_val.shape)

x_train (8516928, 9, 9, 5)
x_test (1216704, 9, 9, 5)
x_val (3650112, 9, 9, 5)
y_train (8516928, 33)
y_test (1216704, 33)
y_val (3650112, 33)


# 定义网络结构

In [72]:
# # 创建神经网络模型
# def create_model():   
#     model = models.Sequential([
#         layers.Dense(200, activation='tanh', input_shape=(5,)),  # 输入层和隐藏层
#         layers.Dense(15, activation='tanh'),
#         layers.Dense(8, activation='tanh'),
#         layers.Dense(1, activation='linear')  # 输出层，num_depths表示深度的数量
#     ])
#     model.compile(optimizer='adam', loss='mean_squared_error')
#     return model 
# #model.summary()

In [74]:
def ChannelAttention(inputs,in_planes, ratio=2):
    
    avg_out= layers.GlobalAveragePooling2D()
    max_out= layers.GlobalMaxPooling2D()

    fc1 = layers.Dense(in_planes//ratio, kernel_initializer='he_normal',
                            kernel_regularizer=regularizers.l2(5e-4),
                            activation=tf.nn.relu,
                            use_bias=True, bias_initializer='zeros')
    fc2 = layers.Dense(in_planes, kernel_initializer='he_normal',
                            kernel_regularizer=regularizers.l2(5e-4),
                            use_bias=True, bias_initializer='zeros')

    
    avg_out = avg_out(inputs)
    max_out = max_out(inputs)
    out = tf.stack([avg_out, max_out], axis=1)  # shape=(None, 2, fea_num)
    out = fc2(fc1(out))
    out = tf.reduce_sum(out, axis=1)             # shape=(256, 512)
    out = tf.nn.sigmoid(out)
    out = layers.Reshape((1, 1, out.shape[1]))(out)

    return  out*inputs

In [75]:
def regularized_padded_conv(*args, **kwargs):
    return layers.Conv2D(*args, **kwargs, padding='same', use_bias=False,
                         kernel_initializer='he_normal',
                         kernel_regularizer=regularizers.l2(5e-4))

def SpatialAttention(inputs,kernel_size=5):
    conv1 = regularized_padded_conv(1, kernel_size=kernel_size, strides=1, activation='sigmoid')
    avg_out = tf.reduce_mean(inputs, axis=3)
    max_out = tf.reduce_max(inputs, axis=3)
    out = tf.stack([avg_out, max_out], axis=-1)             # 创建一个维度,拼接到一起concat。
    out = conv1(out)
    return out*inputs

In [76]:
def cbam_block(inputs, in_planes,ratio=2):

    cbam_feature = ChannelAttention(inputs, in_planes,ratio)
    cbam_feature = SpatialAttention(cbam_feature)
    return cbam_feature

In [77]:
def create_model():
    model_input= Input(shape=x_train.shape[1:])
    output_size = 1
    x = cbam_block(model_input,5)
    x = Conv2D(filters=16, kernel_size=(3, 3),strides=1)(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = cbam_block(x,16)
    x = Conv2D(filters=32, kernel_size=(3, 3),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = cbam_block(x,32)
    x = Flatten()(x)

    x = Dense(units=16)(x)
    x = Activation('relu')(x)

    out = Dense(units=output_size)(x)
    
    model = Model(inputs=model_input, outputs=out)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model 

In [78]:
model = create_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 9, 9, 5)]    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 7, 7, 16)     736         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 7, 7, 16)     64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation (Activation)         (None, 7, 7, 16)     0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [76]:
# initial_learning_rate = 0.0005
# optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)


# model.compile(optimizer='adam', loss='mean_squared_error')
# checkpoint_path='./CNN_model_30.h5'
# keras_callbacks   = [
#       EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
#       ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
# ]

In [77]:
# num_epochs = 1000
# batch_size = 10240

# history = model.fit(x_train, y_train[:,14:15], validation_data=(x_val,y_val[:,14:15]),
#                     epochs=num_epochs, batch_size=batch_size, verbose=2, callbacks=keras_callbacks)

## 网格搜索交叉验证寻找最佳参数

In [78]:
# model = KerasRegressor(build_fn=create_model,verbose=2)

In [79]:
# param_grid = {
#     'neurons1': [16,32,64,128],
#     'neurons2': [32,64,128,256],
# }

In [80]:
# # 使用GridSearchCV进行交叉验证并搜索最佳参数组合：
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
# grid_result = grid_search.fit(x_train, y_train[:,14:15],epochs=100,verbose=2,batch_size=10240)

In [81]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## 训练

In [80]:
def train_model_CNN(x_train,y_train,x_val,y_val,name):
    # 创建网络
    model = create_model()
    # 编译网络
    model.compile(loss='mse', optimizer='adam')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
                    epochs=1000, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [83]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]
# 训练30 - 1000m
for i in range(0,33):
    train_model_CNN(x_train,y_train[:,i:i+1],x_val,y_val[:,i:i+1],
                'D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/CNN/temp/model1/CNN_temp_'+str(depth[i])+'m.h5')

Epoch 1/1000
832/832 - 33s - loss: 0.0524 - val_loss: 0.0139
Epoch 2/1000
832/832 - 25s - loss: 0.0091 - val_loss: 0.0044
Epoch 3/1000
832/832 - 25s - loss: 0.0044 - val_loss: 0.0029
Epoch 4/1000
832/832 - 25s - loss: 0.0034 - val_loss: 0.0024
Epoch 5/1000
832/832 - 25s - loss: 0.0030 - val_loss: 0.0027
Epoch 6/1000
832/832 - 25s - loss: 0.0028 - val_loss: 0.0036
Epoch 7/1000
832/832 - 25s - loss: 0.0026 - val_loss: 0.0022
Epoch 8/1000
832/832 - 25s - loss: 0.0025 - val_loss: 0.0020
Epoch 9/1000
832/832 - 25s - loss: 0.0023 - val_loss: 0.0023
Epoch 10/1000
832/832 - 25s - loss: 0.0023 - val_loss: 0.0019
Epoch 11/1000
832/832 - 25s - loss: 0.0022 - val_loss: 0.0017
Epoch 12/1000
832/832 - 25s - loss: 0.0022 - val_loss: 0.0018
Epoch 13/1000
832/832 - 25s - loss: 0.0021 - val_loss: 0.0016
Epoch 14/1000
832/832 - 25s - loss: 0.0020 - val_loss: 0.0019
Epoch 15/1000
832/832 - 25s - loss: 0.0020 - val_loss: 0.0018
Epoch 16/1000
832/832 - 25s - loss: 0.0020 - val_loss: 0.0020
Epoch 17/1000
832

Epoch 37/1000
832/832 - 25s - loss: 0.0025 - val_loss: 0.0049
Epoch 38/1000
832/832 - 25s - loss: 0.0024 - val_loss: 0.0048
Epoch 39/1000
832/832 - 25s - loss: 0.0025 - val_loss: 0.0032
Epoch 40/1000
832/832 - 25s - loss: 0.0024 - val_loss: 0.0044
Epoch 41/1000
832/832 - 25s - loss: 0.0024 - val_loss: 0.0045
Epoch 42/1000
832/832 - 25s - loss: 0.0025 - val_loss: 0.0039
Epoch 43/1000
832/832 - 25s - loss: 0.0024 - val_loss: 0.0042
Epoch 1/1000
832/832 - 32s - loss: 0.0576 - val_loss: 0.0212
Epoch 2/1000
832/832 - 26s - loss: 0.0155 - val_loss: 0.0098
Epoch 3/1000
832/832 - 26s - loss: 0.0088 - val_loss: 0.0071
Epoch 4/1000
832/832 - 26s - loss: 0.0065 - val_loss: 0.0065
Epoch 5/1000
832/832 - 26s - loss: 0.0055 - val_loss: 0.0052
Epoch 6/1000
832/832 - 26s - loss: 0.0051 - val_loss: 0.0043
Epoch 7/1000
832/832 - 26s - loss: 0.0048 - val_loss: 0.0047
Epoch 8/1000
832/832 - 26s - loss: 0.0046 - val_loss: 0.0037
Epoch 9/1000
832/832 - 26s - loss: 0.0045 - val_loss: 0.0038
Epoch 10/1000
832

Epoch 3/1000
832/832 - 25s - loss: 0.0176 - val_loss: 0.0147
Epoch 4/1000
832/832 - 25s - loss: 0.0155 - val_loss: 0.0139
Epoch 5/1000
832/832 - 25s - loss: 0.0145 - val_loss: 0.0126
Epoch 6/1000
832/832 - 25s - loss: 0.0138 - val_loss: 0.0131
Epoch 7/1000
832/832 - 25s - loss: 0.0134 - val_loss: 0.0124
Epoch 8/1000
832/832 - 25s - loss: 0.0131 - val_loss: 0.0121
Epoch 9/1000
832/832 - 25s - loss: 0.0128 - val_loss: 0.0120
Epoch 10/1000
832/832 - 25s - loss: 0.0126 - val_loss: 0.0134
Epoch 11/1000
832/832 - 25s - loss: 0.0124 - val_loss: 0.0120
Epoch 12/1000
832/832 - 25s - loss: 0.0122 - val_loss: 0.0125
Epoch 13/1000
832/832 - 25s - loss: 0.0120 - val_loss: 0.0118
Epoch 14/1000
832/832 - 25s - loss: 0.0119 - val_loss: 0.0117
Epoch 15/1000
832/832 - 25s - loss: 0.0118 - val_loss: 0.0116
Epoch 16/1000
832/832 - 25s - loss: 0.0117 - val_loss: 0.0137
Epoch 17/1000
832/832 - 25s - loss: 0.0117 - val_loss: 0.0114
Epoch 18/1000
832/832 - 25s - loss: 0.0116 - val_loss: 0.0110
Epoch 19/1000
8

Epoch 47/1000
832/832 - 25s - loss: 0.0125 - val_loss: 0.0125
Epoch 48/1000
832/832 - 25s - loss: 0.0125 - val_loss: 0.0127
Epoch 49/1000
832/832 - 25s - loss: 0.0124 - val_loss: 0.0124
Epoch 50/1000
832/832 - 25s - loss: 0.0124 - val_loss: 0.0132
Epoch 51/1000
832/832 - 25s - loss: 0.0124 - val_loss: 0.0125
Epoch 52/1000
832/832 - 25s - loss: 0.0123 - val_loss: 0.0130
Epoch 53/1000
832/832 - 25s - loss: 0.0124 - val_loss: 0.0128
Epoch 54/1000
832/832 - 25s - loss: 0.0123 - val_loss: 0.0127
Epoch 55/1000
832/832 - 25s - loss: 0.0123 - val_loss: 0.0129
Epoch 56/1000
832/832 - 25s - loss: 0.0123 - val_loss: 0.0138
Epoch 57/1000
832/832 - 25s - loss: 0.0123 - val_loss: 0.0120
Epoch 58/1000
832/832 - 25s - loss: 0.0123 - val_loss: 0.0123
Epoch 59/1000
832/832 - 25s - loss: 0.0123 - val_loss: 0.0126
Epoch 60/1000
832/832 - 25s - loss: 0.0122 - val_loss: 0.0122
Epoch 61/1000
832/832 - 25s - loss: 0.0123 - val_loss: 0.0141
Epoch 62/1000
832/832 - 25s - loss: 0.0123 - val_loss: 0.0126
Epoch 63

Epoch 14/1000
832/832 - 25s - loss: 0.0180 - val_loss: 0.0171
Epoch 15/1000
832/832 - 25s - loss: 0.0179 - val_loss: 0.0175
Epoch 16/1000
832/832 - 25s - loss: 0.0177 - val_loss: 0.0166
Epoch 17/1000
832/832 - 25s - loss: 0.0175 - val_loss: 0.0163
Epoch 18/1000
832/832 - 25s - loss: 0.0172 - val_loss: 0.0178
Epoch 19/1000
832/832 - 25s - loss: 0.0171 - val_loss: 0.0173
Epoch 20/1000
832/832 - 25s - loss: 0.0169 - val_loss: 0.0157
Epoch 21/1000
832/832 - 25s - loss: 0.0168 - val_loss: 0.0165
Epoch 22/1000
832/832 - 25s - loss: 0.0166 - val_loss: 0.0159
Epoch 23/1000
832/832 - 25s - loss: 0.0165 - val_loss: 0.0160
Epoch 24/1000
832/832 - 25s - loss: 0.0164 - val_loss: 0.0155
Epoch 25/1000
832/832 - 25s - loss: 0.0163 - val_loss: 0.0152
Epoch 26/1000
832/832 - 25s - loss: 0.0162 - val_loss: 0.0154
Epoch 27/1000
832/832 - 25s - loss: 0.0161 - val_loss: 0.0151
Epoch 28/1000
832/832 - 25s - loss: 0.0161 - val_loss: 0.0171
Epoch 29/1000
832/832 - 25s - loss: 0.0160 - val_loss: 0.0160
Epoch 30

...

832/832 - 25s - loss: 0.0232 - val_loss: 0.0254
Epoch 46/1000
832/832 - 25s - loss: 0.0232 - val_loss: 0.0243
Epoch 47/1000
832/832 - 25s - loss: 0.0231 - val_loss: 0.0249
Epoch 48/1000
832/832 - 25s - loss: 0.0231 - val_loss: 0.0255
Epoch 49/1000
832/832 - 25s - loss: 0.0231 - val_loss: 0.0252
Epoch 50/1000
832/832 - 25s - loss: 0.0230 - val_loss: 0.0255
Epoch 51/1000
832/832 - 25s - loss: 0.0230 - val_loss: 0.0253
Epoch 52/1000
832/832 - 25s - loss: 0.0229 - val_loss: 0.0245
Epoch 53/1000
832/832 - 25s - loss: 0.0228 - val_loss: 0.0243
Epoch 54/1000
832/832 - 25s - loss: 0.0228 - val_loss: 0.0240
Epoch 55/1000
832/832 - 25s - loss: 0.0228 - val_loss: 0.0251
Epoch 56/1000
832/832 - 25s - loss: 0.0227 - val_loss: 0.0252
Epoch 1/1000
832/832 - 29s - loss: 0.1054 - val_loss: 0.0528
Epoch 2/1000
832/832 - 25s - loss: 0.0450 - val_loss: 0.0364
Epoch 3/1000
832/832 - 25s - loss: 0.0359 - val_loss: 0.0309
Epoch 4/1000
832/832 - 25s - loss: 0.0321 - val_loss: 0.0304
Epoch 5/1000
832/832 - 25s

Epoch 57/1000
832/832 - 26s - loss: 0.0212 - val_loss: 0.0215
Epoch 58/1000
832/832 - 26s - loss: 0.0212 - val_loss: 0.0219
Epoch 59/1000
832/832 - 26s - loss: 0.0212 - val_loss: 0.0213
Epoch 60/1000
832/832 - 26s - loss: 0.0211 - val_loss: 0.0213
Epoch 61/1000
832/832 - 26s - loss: 0.0211 - val_loss: 0.0221
Epoch 62/1000
832/832 - 26s - loss: 0.0211 - val_loss: 0.0220
Epoch 63/1000
832/832 - 26s - loss: 0.0210 - val_loss: 0.0219
Epoch 64/1000
832/832 - 26s - loss: 0.0210 - val_loss: 0.0216
Epoch 65/1000
832/832 - 26s - loss: 0.0210 - val_loss: 0.0212
Epoch 66/1000
832/832 - 26s - loss: 0.0210 - val_loss: 0.0217
Epoch 67/1000
832/832 - 26s - loss: 0.0210 - val_loss: 0.0215
Epoch 68/1000
832/832 - 26s - loss: 0.0209 - val_loss: 0.0215
Epoch 69/1000
832/832 - 26s - loss: 0.0209 - val_loss: 0.0216
Epoch 70/1000
832/832 - 26s - loss: 0.0209 - val_loss: 0.0209
Epoch 71/1000
832/832 - 26s - loss: 0.0209 - val_loss: 0.0215
Epoch 72/1000
832/832 - 26s - loss: 0.0209 - val_loss: 0.0216
Epoch 73

Epoch 62/1000
832/832 - 25s - loss: 0.0254 - val_loss: 0.0254
Epoch 63/1000
832/832 - 25s - loss: 0.0253 - val_loss: 0.0254
Epoch 64/1000
832/832 - 25s - loss: 0.0253 - val_loss: 0.0253
Epoch 65/1000
832/832 - 25s - loss: 0.0253 - val_loss: 0.0251
Epoch 66/1000
832/832 - 25s - loss: 0.0252 - val_loss: 0.0249
Epoch 67/1000
832/832 - 25s - loss: 0.0252 - val_loss: 0.0265
Epoch 68/1000
832/832 - 25s - loss: 0.0252 - val_loss: 0.0266
Epoch 69/1000
832/832 - 25s - loss: 0.0252 - val_loss: 0.0245
Epoch 70/1000
832/832 - 25s - loss: 0.0252 - val_loss: 0.0246
Epoch 71/1000
832/832 - 25s - loss: 0.0251 - val_loss: 0.0257
Epoch 72/1000
832/832 - 25s - loss: 0.0250 - val_loss: 0.0254
Epoch 73/1000
832/832 - 25s - loss: 0.0250 - val_loss: 0.0245
Epoch 74/1000
832/832 - 25s - loss: 0.0250 - val_loss: 0.0250
Epoch 75/1000
832/832 - 25s - loss: 0.0250 - val_loss: 0.0260
Epoch 76/1000
832/832 - 25s - loss: 0.0250 - val_loss: 0.0246
Epoch 77/1000
832/832 - 25s - loss: 0.0249 - val_loss: 0.0248
Epoch 78

Epoch 17/1000
832/832 - 25s - loss: 0.0329 - val_loss: 0.0326
Epoch 18/1000
832/832 - 25s - loss: 0.0324 - val_loss: 0.0333
Epoch 19/1000
832/832 - 25s - loss: 0.0319 - val_loss: 0.0323
Epoch 20/1000
832/832 - 25s - loss: 0.0314 - val_loss: 0.0309
Epoch 21/1000
832/832 - 25s - loss: 0.0311 - val_loss: 0.0301
Epoch 22/1000
832/832 - 25s - loss: 0.0308 - val_loss: 0.0321
Epoch 23/1000
832/832 - 25s - loss: 0.0305 - val_loss: 0.0308
Epoch 24/1000
832/832 - 25s - loss: 0.0301 - val_loss: 0.0323
Epoch 25/1000
832/832 - 25s - loss: 0.0300 - val_loss: 0.0282
Epoch 26/1000
832/832 - 25s - loss: 0.0296 - val_loss: 0.0284
Epoch 27/1000
832/832 - 25s - loss: 0.0293 - val_loss: 0.0296
Epoch 28/1000
832/832 - 25s - loss: 0.0290 - val_loss: 0.0301
Epoch 29/1000
832/832 - 25s - loss: 0.0289 - val_loss: 0.0303
Epoch 30/1000
832/832 - 25s - loss: 0.0287 - val_loss: 0.0286
Epoch 31/1000
832/832 - 25s - loss: 0.0285 - val_loss: 0.0289
Epoch 32/1000
832/832 - 25s - loss: 0.0281 - val_loss: 0.0290
Epoch 33

Epoch 59/1000
832/832 - 26s - loss: 0.0382 - val_loss: 0.0379
Epoch 60/1000
832/832 - 26s - loss: 0.0380 - val_loss: 0.0379
Epoch 61/1000
832/832 - 26s - loss: 0.0382 - val_loss: 0.0375
Epoch 62/1000
832/832 - 26s - loss: 0.0381 - val_loss: 0.0378
Epoch 63/1000
832/832 - 26s - loss: 0.0377 - val_loss: 0.0375
Epoch 64/1000
832/832 - 26s - loss: 0.0379 - val_loss: 0.0372
Epoch 65/1000
832/832 - 26s - loss: 0.0380 - val_loss: 0.0377
Epoch 66/1000
832/832 - 26s - loss: 0.0376 - val_loss: 0.0405
Epoch 67/1000
832/832 - 26s - loss: 0.0374 - val_loss: 0.0370
Epoch 68/1000
832/832 - 26s - loss: 0.0376 - val_loss: 0.0380
Epoch 69/1000
832/832 - 26s - loss: 0.0373 - val_loss: 0.0377
Epoch 70/1000
832/832 - 26s - loss: 0.0374 - val_loss: 0.0386
Epoch 71/1000
832/832 - 26s - loss: 0.0371 - val_loss: 0.0358
Epoch 72/1000
832/832 - 26s - loss: 0.0372 - val_loss: 0.0371
Epoch 73/1000
832/832 - 26s - loss: 0.0371 - val_loss: 0.0372
Epoch 74/1000
832/832 - 26s - loss: 0.0369 - val_loss: 0.0392
Epoch 75

In [ ]:
# model = create_model(128,256)
# model.compile(optimizer='adam', loss='mean_squared_error')
# checkpoint_path='./CNN_model_16.h5'
# keras_callbacks   = [
#       EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.001),
#       ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
# ]

In [ ]:
# num_epochs = 1000
# batch_size = 10240

# history = model.fit(x_train, y_train[:,14:15], validation_data=(x_val,y_val[:,14:15]),
#                     epochs=num_epochs, batch_size=batch_size, verbose=2, callbacks=keras_callbacks)

## 模型评估

In [95]:
# 评估函数定义
# 异常相关系数
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)


# 查看损失
def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()


# 模型预测
def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    a = y_test
    y_test_p = a.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)


    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)
    #print('%.3f' % rmse)

   # print('acc:',acc(y_test_p,testPred_p))

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score: %.4f" % r2)  
   # print("%.4f"% r2)  
    

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        # 模型预测
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

In [97]:
#look_loss(history)

### 对指定层进行评估

In [98]:
# print(x_test.shape,y_test.shape)

In [99]:
# best_model = tf.keras.models.load_model('CNN_model_16.h5')
# all_estimate(best_model,x_test,y_test[:,14:15])

### 对所有层进行评估

In [102]:
# 模型预测
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/CNN/temp/model1/CNN_temp_'+str(depth[i])+'m.h5')
    print(str(depth[i])+'m 测试集：')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
# re2_list,rmse_list = respective_estimate(best_model,x_test,y_test[:,:,:,:,0:5])
# re2_list,rmse_list

5m 测试集：
Test RMSE: 0.037
R² score: 0.9986
10m 测试集：
Test RMSE: 0.041
R² score: 0.9983
15m 测试集：
Test RMSE: 0.054
R² score: 0.9971
20m 测试集：
Test RMSE: 0.059
R² score: 0.9965
30m 测试集：
Test RMSE: 0.089
R² score: 0.9921
40m 测试集：
Test RMSE: 0.110
R² score: 0.9877
50m 测试集：
Test RMSE: 0.131
R² score: 0.9823
60m 测试集：
Test RMSE: 0.142
R² score: 0.9786
70m 测试集：
Test RMSE: 0.146
R² score: 0.9767
80m 测试集：
Test RMSE: 0.172
R² score: 0.9669
90m 测试集：
Test RMSE: 0.163
R² score: 0.9697
100m 测试集：
Test RMSE: 0.164
R² score: 0.9688
125m 测试集：
Test RMSE: 0.178
R² score: 0.9622
150m 测试集：
Test RMSE: 0.185
R² score: 0.9593
175m 测试集：
Test RMSE: 0.181
R² score: 0.9617
200m 测试集：
Test RMSE: 0.179
R² score: 0.9637
225m 测试集：
Test RMSE: 0.169
R² score: 0.9684
250m 测试集：
Test RMSE: 0.173
R² score: 0.9677
275m 测试集：
Test RMSE: 0.168
R² score: 0.9699
300m 测试集：
Test RMSE: 0.170
R² score: 0.9697
350m 测试集：
Test RMSE: 0.182
R² score: 0.9662
400m 测试集：
Test RMSE: 0.191
R² score: 0.9634
500m 测试集：
Test RMSE: 0.193
R² score: 0.9630


# 保存预测结果

In [103]:
# 模型预测
result = [] 
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/CNN/temp/model1/CNN_temp_'+str(depth[i])+'m.h5')
    #print('预测第'+str(depth[i]+'m的温度')
    testPred = best_model.predict(x_test)
    result.append(testPred)

In [104]:
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,33)
print(result.shape)

(33, 1216704, 1)
(1216704, 1, 33)
(1216704, 33)


In [105]:
#np.save("model/CNN/result/result_CNN_CBAM_30_1000m.npy",result)
# result= np.load("model/CNN/result/result_CNN_CBAM_30_1000m.npy")

## 对预测结果进行反归一化

In [84]:
#y_test = y_test.reshape(-1,15)

In [108]:
result.shape,y_test.shape 

((1216704, 33), (1216704, 33))

In [109]:
result_unscaled = scaler_l.inverse_transform(result)
y_test_unscaled = scaler_l.inverse_transform(y_test)

In [110]:
result_unscaled.shape,y_test_unscaled.shape 

((1216704, 33), (1216704, 33))

## 评估反归一化之后的结果

In [111]:
for i in range(0,33):
    r2 = r2_score(y_test_unscaled[:,i],result_unscaled[:,i])
    print(str(depth[i])+'m R² score:',r2)

5m R² score: 0.9985896326710891
10m R² score: 0.9982757360782517
15m R² score: 0.9970719169214098
20m R² score: 0.996487482247341
30m R² score: 0.992056257475088
40m R² score: 0.9876849710729833
50m R² score: 0.9822870224365073
60m R² score: 0.9786403129121505
70m R² score: 0.9766988751609851
80m R² score: 0.9668860836506968
90m R² score: 0.9697372752403235
100m R² score: 0.9687788222064719
125m R² score: 0.9621824862744751
150m R² score: 0.9593204922640667
175m R² score: 0.9616965355843076
200m R² score: 0.9637045752439727
225m R² score: 0.968414446296587
250m R² score: 0.9677128195557194
275m R² score: 0.9698990029293602
300m R² score: 0.9697267626290519
350m R² score: 0.9662430194711149
400m R² score: 0.9633746565033106
500m R² score: 0.9630416544024337
600m R² score: 0.9609044308229127
700m R² score: 0.9649400816795359
800m R² score: 0.9684546233462398
900m R² score: 0.9699589374450002
1000m R² score: 0.970413308676669
1100m R² score: 0.9672736620696382
1300m R² score: 0.9647485271

In [112]:
for i in range(0,33):
    mse =  mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i])
    print(str(depth[i])+'m Test mse: %.3f' % mse)

5m Test mse: 0.094
10m Test mse: 0.115
15m Test mse: 0.197
20m Test mse: 0.237
30m Test mse: 0.546
40m Test mse: 0.863
50m Test mse: 1.234
60m Test mse: 1.443
70m Test mse: 1.496
80m Test mse: 2.011
90m Test mse: 1.728
100m Test mse: 1.671
125m Test mse: 1.670
150m Test mse: 1.444
175m Test mse: 1.085
200m Test mse: 0.840
225m Test mse: 0.619
250m Test mse: 0.552
275m Test mse: 0.459
300m Test mse: 0.416
350m Test mse: 0.376
400m Test mse: 0.323
500m Test mse: 0.187
600m Test mse: 0.104
700m Test mse: 0.052
800m Test mse: 0.030
900m Test mse: 0.021
1000m Test mse: 0.016
1100m Test mse: 0.013
1300m Test mse: 0.009
1500m Test mse: 0.007
1750m Test mse: 0.003
2000m Test mse: 0.004


# 预测结果可视化

## 为归一化之后的预测值和真实值填充nan

In [114]:
len(nan_mask)

161024

In [ ]:
# cat_data_ss_depth_test = np.transpose(cat_data_ss_depth,(1,2,0,3))
# print(cat_data_ss_depth_test.shape)
# cat_data_ss_depth_test = cat_data_ss_depth_test.reshape ((-1,132*20))
# print(cat_data_ss_depth_test.shape)

In [ ]:
# cat_data_ss_depth_test[cat_data_ss_depth_test == 32767] = np.nan
# print('cat_data_ss_depth_test删除nan值之前',cat_data_ss_depth_test.shape)
# # 使用 numpy.isnan 函数检测NaN值
# nan_mask = np.isnan(cat_data_ss_depth_test)

In [ ]:
# nan_mask[nan_mask.any(axis=1) == True] = True   # nan为True,只要有一个通道存在nan,就都设置为nan
# nan_masks = nan_mask[:,1]    #得到图像的masks

In [115]:
### 创建一个形状与nan_masks相同且全是nan的数组
testPred_with_nan = np.full((161024,12*33), np.nan)  
print(testPred_with_nan.shape)

y_test_with_nan = np.full((161024,12*33), np.nan)  
print(y_test_with_nan.shape)

(161024, 396)
(161024, 396)


In [116]:
result_unscaled_reshape = result_unscaled.reshape(12,-1,33)
print(result_unscaled_reshape.shape)
result_unscaled_reshape = np.transpose(result_unscaled_reshape,(1,0,2))
print(result_unscaled_reshape.shape)
result_unscaled_reshape = result_unscaled_reshape.reshape(-1,12*33)
print(result_unscaled_reshape.shape)

# ==================================================================== #

y_test_unscaled_reshape = y_test_unscaled.reshape(12,-1,33)
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = np.transpose(y_test_unscaled_reshape,(1,0,2))
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = y_test_unscaled_reshape.reshape(-1,12*33)
print(y_test_unscaled_reshape.shape)

(12, 101392, 33)
(101392, 12, 33)
(101392, 396)
(12, 101392, 33)
(101392, 12, 33)
(101392, 396)


In [117]:
y_test_unscaled.shape 

(1216704, 33)

In [118]:
num = 0
for i,nanmask in enumerate(nan_mask):   # i从0开始

    if (nanmask ==True): # True 代表没有nan，使用预测值进行填充
        testPred_with_nan[i] = result_unscaled_reshape[num]
        y_test_with_nan[i] = y_test_unscaled_reshape[num]
        num = num + 1
        
print(num)

101392


In [119]:
testPred_with_nan = testPred_with_nan.reshape(272, 592,12,33)
testPred_with_nan = np.transpose(testPred_with_nan,(2,3,0,1))


y_test_with_nan = y_test_with_nan.reshape(272, 592,12,33)
y_test_with_nan = np.transpose(y_test_with_nan,(2,3,0,1))

print(testPred_with_nan.shape,y_test_with_nan.shape)

(12, 33, 272, 592) (12, 33, 272, 592)


# 结果可视化

In [120]:
lat = []
lon = []
time = []

In [121]:
lon.append(data_ssh['lon'][4:-4].data)
lat.append(data_ssh['lat'][4:-4].data)

In [122]:
len(lat),len(lat[0]),len(lon),len(lon[0])

(1, 272, 1, 592)

In [123]:
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [124]:
time 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [128]:
# new_NC.close()

In [126]:
new_NC = nc.Dataset("D:/codeFile/jupyterDemo/3d_ts/BPNN/model/new_0-2000m_24.6.6/CNN/temp/result/model1/result5_2000m_CNN_CBAM.nc",
                    'w', format='NETCDF4')

'''
定义维度，后一个参数表示维度的长度，因为是合并的同一个产品的数据，所以是统一
的,注意维度的长度一定要和读入的数据匹配
'''

new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
# new_NC.createDimension('Observed_ST', len(y_test_with_nan))
# new_NC.createDimension('Estimated_ST', len(testPred_with_nan))

new_NC.createDimension('depth', 33)
new_NC.createDimension('time', len(time))

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f', ("depth"))
new_NC.createVariable('Observed_ST', 'f',("time","depth","lat","lon"))
new_NC.createVariable('Estimated_ST', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'

#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['depth'][:] = depth

new_NC.variables['Observed_ST'][:]=np.array(y_test_with_nan)
new_NC.variables['Estimated_ST'][:]=np.array(testPred_with_nan)


#最后记得关闭文件
new_NC.close()


In [ ]:
depth